In [2]:
import os
import botocore
from boto3.session import Session
import time
import throttle

# 准备密钥
aws_access_key_id = 'hust'
aws_secret_access_key = 'hust_obs'

# 建立会话
session = Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# 连接到服务
s3 = session.resource('s3', endpoint_url='http://192.168.33.1:9000')

查看所有bucket

In [3]:
for bucket in s3.buckets.all():
    print('bucket name:%s' % bucket.name)

bucket name:loadgen
bucket name:test001


新建一个bucket (注意："bucket name" 中不能有下划线)

In [4]:
bucket_name = 'test100objs'
if s3.Bucket(bucket_name) not in s3.buckets.all():
    s3.create_bucket(Bucket=bucket_name)

查看此bucket下的所有object

In [5]:
bucket = s3.Bucket(bucket_name)
for obj in bucket.objects.all():
    print('obj name:%s' % obj.key)

准备负载，按照几种不同请求到达率(Inter-Arrival Time, IAT)设置。

In [7]:
local_file = "_test_4K.bin"
test_bytes = [0xFF for i in range(1024*4)]

with open(local_file, "wb") as lf:
    lf.write(bytearray(test_bytes))

# 发起请求和计算系统停留时间
def request_timing(i):
    # 上传obj
    obj_name = "testObj%08d"%(i,)
    # temp_file = '.tempfile'
    service_time = 0
    start = time.time()
    s3.Object(bucket_name, obj_name).upload_file(local_file)
    # 或
    # bucket.put_object(Key=obj_name, Body=open(local_file, 'rb'))
    # 下载obj
    # s3.Object(bucket_name, obj_name).download_file(temp_file)
    end = time.time()
    system_time = end - start
    return system_time * 1000 # 换算为毫秒

# 按照请求到达率限制来执行和跟踪请求
def arrival_rate_max(i): # unlimited
    return request_timing(i)

@throttle.wrap(0.1, 2) # 100ms 2 个
def arrival_rate_2(i):
    return request_timing(i)

@throttle.wrap(0.1, 4)
def arrival_rate_4(i):
    return request_timing(i)

@throttle.wrap(0.1, 8)
def arrival_rate_8(i):
    return request_timing(i)

按照预设IAT发起请求

In [8]:
latency = []
for i in range(100):
    # 上传obj
    st = arrival_rate_max(i)
    succ = not st is throttle.fail
    print(succ, st)
    if succ:
        latency.append(st)

True 19.048452377319336
True 18.268585205078125
True 16.484737396240234
True 19.948482513427734
True 20.379066467285156
True 20.626306533813477
True 19.40321922302246
True 16.15309715270996
True 16.213417053222656
True 19.46878433227539
True 12.932777404785156
True 13.104677200317383
True 12.191295623779297
True 12.163162231445312
True 18.07093620300293
True 16.010522842407227
True 13.829231262207031
True 14.847755432128906
True 11.795520782470703
True 12.924909591674805
True 13.800859451293945
True 11.71875
True 12.505292892456055
True 12.422323226928711
True 12.597799301147461
True 13.301610946655273
True 12.873172760009766
True 45.21822929382324
True 15.040159225463867
True 12.68458366394043
True 11.396169662475586
True 13.08298110961914
True 12.254476547241211
True 11.559486389160156
True 13.336896896362305
True 12.578487396240234
True 13.459920883178711
True 14.1754150390625
True 12.149333953857422
True 13.643741607666016
True 17.397165298461914
True 16.630887985229492
True 13.532

清理

In [9]:
try:
    # 删除bucket下所有object
    bucket.objects.filter().delete()

    # 删除bucket下某个object
    # bucket.objects.filter(Prefix=obj_name).delete()

    bucket.delete()
except botocore.exceptions.ClientError as e:
    print('error in bucket removal')

In [10]:
# 删除本地测试文件

os.remove(local_file)

记录延迟到CSV文件

In [11]:
with open("latency.csv", "w+") as tracefile:
    tracefile.write("latency\n")
    tracefile.writelines([str(l) + '\n' for l in latency])